
### LoRA 是什么？

是一种不直接修改预训练权重（$W$），仅通过低秩增量（$\Delta W$）调整模型效果的微调方法，公式为$W' = W + \Delta W = W + \frac{\alpha}{r}A \times B$。


其中$\alpha$ 用于控制对原始模型权重更新的强度，过大，会很快适应新任务，但也可能导致过拟合。

### lora的优点？
1. 微调效率高

2. 保留了原始模型的能力。直接修改原始模型W可能会导致在原始任务上灾难性遗忘，lora通过冻结，保留了原始模型的能力。

3. 快速切换。任务切换只需要加载不同Lora文件，方便快速。

### Lora为什么有效？

大模型预训练权重已经包含了通用知识，微调时只需要对权重矩阵做任务特定的稀疏调整（ΔW）。LoRA通过低秩矩阵A和B来逼近这个更新（ΔW≈A·B），这类似于用可训练参数实现了一种数据驱动的低秩近似，而不需要显式计算SVD分解。


### LoRA 矩阵初始化方式?
下采样矩阵A为随机高斯初始化（非零）；上采样矩阵 B为全零初始化。

如此实现，$\Delta W = A \cdot B = 0$（因 B 全零），模型初始权重 = 预训练权重 $W$。


- A和B可以都为0吗？ 不可以，因为那样会导致梯度都为0，无法更新。
- 可以反过来吗？ 有篇论文（The Impact of Initialization on LoRA Finetuning Dynamics）证明了可以，但是A初始有值效果会更好。



## lora代码实现

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank=4, alpha=8):
        super().__init__()
        self.alpha = alpha
        self.rank = rank
        # A: 随机高斯初始化（标准差=1/√rank）
        self.A = nn.Parameter(torch.randn(in_dim, rank) / torch.sqrt(torch.tensor(rank)))
        # B: 全零初始化
        self.B = nn.Parameter(torch.zeros(rank, out_dim))

    def forward(self, x):
        # ΔW = (A·B) * (alpha/rank)，缩放保证训练稳定性
        return self.alpha / self.rank * (x @ self.A @ self.B)

class LinearWithLoRA(nn.Module):
    def __init__(self, linear_layer, rank=4, alpha=8):
        super().__init__()
        self.linear = linear_layer  # 冻结的预训练线性层
        self.lora = LoRALayer(
            in_dim=linear_layer.in_features,
            out_dim=linear_layer.out_features,
            rank=rank,
            alpha=alpha
        )
        for param in self.linear.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.linear(x) + self.lora(x)

# --------------------------- 测试运行 ---------------------------
if __name__ == "__main__":
    # 1. 模拟一个预训练线性层（例如Transformer中的QKV投影层）
    pretrained_linear = nn.Linear(in_features=12, out_features=12)
    # 2. 包装LoRA（rank=4, alpha=8）
    lora_linear = LinearWithLoRA(pretrained_linear, rank=4, alpha=8)
    # 3. 随机输入（batch_size=2, seq_len=10, hidden_dim=128）
    x = torch.randn(2, 10, 12)
    # 4. 前向传播（测试输出形状）
    output = lora_linear(x)
    print(f"输入形状: {x.shape}")       # torch.Size([2, 10, 12])
    print(f"输出形状: {output.shape}")  # torch.Size([2, 10, 12]) （形状匹配）

    # 5. 简化训练逻辑（仅示意参数更新）
    optimizer = optim.Adam(lora_linear.lora.parameters(), lr=1e-4)  # 仅优化LoRA参数
    loss_fn = nn.MSELoss()
    target = torch.randn_like(output)  # 模拟目标值

    # 前向+反向+更新
    optimizer.zero_grad()
    pred = lora_linear(x)
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

    print(f"训练后LoRA B参数示例（非零，说明更新成功）: \n{lora_linear.lora.B[:2, :2]}")

输入形状: torch.Size([2, 10, 12])
输出形状: torch.Size([2, 10, 12])
训练后LoRA B参数示例（非零，说明更新成功）: 
tensor([[-1.0000e-04, -1.0000e-04],
        [-1.0000e-04, -1.0000e-04]], grad_fn=<SliceBackward0>)
